In [1]:
!pip install transformers torch scikit-learn
!pip install sentence-transformers
!pip install pocketsphinx
!pip install SpeechRecognition
!pip install pyaudio
!pip install tf-keras
!pip install pyttsx3
!pip install -U sentence-transformers

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1


In [2]:
import speech_recognition as sr
import pyttsx3 as tts

r = sr.Recognizer()
engine = tts.init()
engine.setProperty('rate',125)

def talking(text):
    engine.say(text)
    engine.runAndWait()

def getText():
    with sr.Microphone() as source:
        try:
            print("Słucham...")
            audio = r.listen(source)
            #text = r.recognize_sphinx(audio, language='pl')
            text = r.recognize_google(audio, language='pl-PL')
            if text != "":
                return text
            return 0
        except:
            return 0
    
while True:
    txt = getText()
    if not txt == 0:
        print(txt)
        talking(txt)
        break
    else:
        print("Nic nie udało się rozpoznać...")
        continue


Słucham...
halo halo ha


In [31]:
# Funkcja do rozpoznawania mowy z mikrofonu
def speech_to_text_offline():
    with sr.Microphone() as source:
        print("Proszę mówić...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        
        try:
            # Użycie pocketsphinx do rozpoznawania mowy offline
            text = recognizer.recognize_sphinx(audio, language='pl')
            print("Rozpoznany tekst:", text)
        except sr.UnknownValueError:
            print("Nie rozpoznano mowy")
        except sr.RequestError as e:
            print(f"Błąd podczas korzystania z pocketsphinx: {e}")

# Wywołanie funkcji
speech_to_text_offline()

Proszę mówić...
Błąd podczas korzystania z pocketsphinx: missing PocketSphinx language data directory: "C:\Users\Sikma\AppData\Roaming\Python\Python312\site-packages\speech_recognition\pocketsphinx-data\pl"


In [4]:
print(txt)

halo halo ha


In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from transformers import AutoModel, AutoTokenizer
import json

import nltk 
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel
import torch

import time
import os

I:\Anaconda\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [11]:
dataset_aborcja = pd.read_csv('datasets/aborcja.csv', encoding='utf-8', sep=';', usecols=["statement", "score"])
dataset_armia_ue = pd.read_csv('datasets/armia_ue.csv', encoding='utf-8', sep=';')
dataset_bron = pd.read_csv('datasets/bron.csv', encoding='utf-8', sep=';')
dataset_cpk = pd.read_csv('datasets/cpk.csv', encoding='utf-8', sep=';')
dataset_dochodowy = pd.read_csv('datasets/dochodowy.csv', encoding='utf-8', sep=';')
dataset_euro = pd.read_csv('datasets/euro.csv', encoding='utf-8', sep=';')
dataset_eutanazja = pd.read_csv('datasets/eutanazja.csv', encoding='utf-8', sep=';')
dataset_imigranci= pd.read_csv('datasets/imigranci.csv', encoding='utf-8', sep=';')
dataset_invitro = pd.read_csv('datasets/invitro.csv', encoding='utf-8', sep=';')
dataset_kara_smierci = pd.read_csv('datasets/kara_smierci.csv', encoding='utf-8', sep=';')
dataset_katastralny = pd.read_csv('datasets/katastralny.csv', encoding='utf-8', sep=';')
dataset_osiemset = pd.read_csv('datasets/osiemset.csv', encoding='utf-8', sep=';')
dataset_samochody = pd.read_csv('datasets/samochody.csv', encoding='utf-8', sep=';')
dataset_sluzba_wojskowa = pd.read_csv('datasets/sluzba_wojskowa.csv', encoding='utf-8', sep=';')
dataset_ue = pd.read_csv('datasets/ue.csv', encoding='utf-8', sep=';')
dataset_wdowia= pd.read_csv('datasets/wdowia.csv', encoding='utf-8', sep=';')
dataset_zus = pd.read_csv('datasets/zus.csv', encoding='utf-8', sep=';')

datasets = {
    "dataset_aborcja": dataset_aborcja,
    "dataset_eutanazja": dataset_eutanazja,
    "dataset_bron": dataset_bron,
    "dataset_kara_smierci": dataset_kara_smierci,
    "dataset_invitro": dataset_invitro,
    "dataset_armia_ue": dataset_armia_ue,
    "dataset_euro": dataset_euro,
    "dataset_cpk": dataset_cpk,
    "dataset_sluzba_wojskowa": dataset_sluzba_wojskowa,
    "dataset_ue": dataset_ue,
    "dataset_zus": dataset_zus,
    "dataset_samochody": dataset_samochody,
    "dataset_osiemset": dataset_osiemset,
    "dataset_dochodowy": dataset_dochodowy,
    "dataset_katastralny": dataset_katastralny,
    "dataset_imigranci": dataset_imigranci,
    "dataset_wdowia": dataset_wdowia
}

In [3]:
dataset_aborcja.dtypes

statement     object
score        float64
dtype: object

In [4]:
for name, dataset in datasets.items(): # iterowanie przez wszystkie pozycje klucz-wartość w słowniku. klucz to nazwy w cudzysłowie, a wartości to sam dataset.
    print(f'Średnia dla {name}:', dataset['score'].mean())
    print(f'Mediana dla {name}:', dataset['score'].median(), '\n')


Średnia dla dataset_aborcja: -0.00466321243523319
Mediana dla dataset_aborcja: -0.1 

Średnia dla dataset_eutanazja: 0.020967741935483883
Mediana dla dataset_eutanazja: 0.0 

Średnia dla dataset_bron: 0.038095238095238085
Mediana dla dataset_bron: 0.1 

Średnia dla dataset_kara_smierci: -0.09594594594594595
Mediana dla dataset_kara_smierci: 0.0 

Średnia dla dataset_invitro: 0.0
Mediana dla dataset_invitro: -0.1 

Średnia dla dataset_armia_ue: 0.09322033898305085
Mediana dla dataset_armia_ue: 0.2 

Średnia dla dataset_euro: 0.014655172413793128
Mediana dla dataset_euro: -0.05 

Średnia dla dataset_cpk: -0.033018867924528315
Mediana dla dataset_cpk: -0.2 

Średnia dla dataset_sluzba_wojskowa: -0.01666666666666667
Mediana dla dataset_sluzba_wojskowa: 0.0 

Średnia dla dataset_ue: -0.028571428571428564
Mediana dla dataset_ue: 0.0 

Średnia dla dataset_zus: 0.018518518518518517
Mediana dla dataset_zus: 0.05 

Średnia dla dataset_samochody: 0.023684210526315797
Mediana dla dataset_samochody

## Wykonanie embeddingu wszystkich datasetów i zapis do plików .csv
przeiterowanie po słowniku datasets, dokonanie embeddingów na każdym z nich oraz zapisanie odpowiednio oczyszczonych dataframeów do plików z rozszerzeniem .csv

In [ ]:
#dla pojedynczego datasetu:
dataset_embeddings = [
    (get_embedding(row["statement"]), row["score"], row["statement"]) 
    for _, row in dataset.iterrows() #embedding każdego datasetu
]  
df_dataset_embeddings = pd.DataFrame(dataset_embeddings, columns=["embedding", "score", "statement"]) #konwersja zestawu danych do dataframe
df_dataset_embeddings["embedding"] = df_dataset_embeddings["embedding"].apply(lambda x: ",".join(map(str, x)))
df_dataset_embeddings.to_csv("dataset_embeddings.csv", index=False, encoding="utf-8")

In [12]:
def get_datasets_embeddings_and_CSVsave():
    for dataset_name, dataset in datasets.items():
        dataset_embeddings = [
            (get_embedding(row["statement"]), row["score"], row["statement"]) 
            for _, row in dataset.iterrows() #embedding każdego datasetu
        ]
        
        df_dataset_embeddings = pd.DataFrame(dataset_embeddings, columns=["embedding", "score", "statement"]) #konwersja zestawu danych do dataframe

        df_dataset_embeddings["embedding"] = df_dataset_embeddings["embedding"].apply(lambda x: ",".join(map(str, x)))
        df_dataset_embeddings.to_csv(f"{dataset_name}_embeddings.csv", index=False, encoding="utf-8") #zapis dataframe do pliku w formacie .csv
        
get_datasets_embeddings_and_CSVsave()

## Załadowanie wszystkich zapisanych datasetów z plików .csv
- załadowanie wszystkich zbiorów w formacie .csv z folderu "datasets_embeddings" i konwersja kolumny "embedding" na poprawną
- wszystkie załadowane pliki są zapisane w zmiennych globalnych o nazwach: "df_"+nazwa datasetu np. df_dataset_aborcja

In [10]:
#dla pojedynczego datasetu
#df_loaded = pd.read_csv("datasets_embeddings/dataset_bron_embeddings.csv", encoding="utf-8")
#df_loaded["embedding"] = df_loaded["embedding"].apply(lambda x: np.array(list(map(float, x.split(",")))))

def CSVload_datasets_embedded(directory):
    
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)

        df = pd.read_csv(filepath, encoding="utf-8")
        df["embedding"] = df["embedding"].apply(lambda x: np.array(list(map(float, x.split(",")))))

        dataset_name = f'df_{filename.replace("_embeddings.csv", "")}'
        
        globals()[dataset_name] = df

CSVload_datasets_embedded("datasets_embeddings")


In [4]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', 'open', '_', '__', '___', '__session__', 'json', 'sys', '__import__', 'NamespaceMagics', '_jupyterlab_variableinspector_nms', '_jupyterlab_variableinspector_Jupyter', '__np', '__pd', '__pyspark', '__tf', '__K', '__torch', '__ipywidgets', '__xr', '_attempt_import', '_check_imported', '_jupyterlab_variableinspector_getsizeof', '_jupyterlab_variableinspector_getshapeof', '_jupyterlab_variableinspector_getcontentof', '_jupyterlab_variableinspector_is_matrix', '_jupyterlab_variableinspector_is_widget', '_jupyterlab_variableinspector_dict_list', '_jupyterlab_variableinspector_getmatrixcontent', '_jupyterlab_variableinspector_displaywidget', '_jupyterlab_variableinspector_default', '_jupyterlab_variableinspector_deletevariable', '_1', '_i', '_ii', '_iii', '_i1', 'SentenceTransformer', 'cosine_similarity', 'np', 'pd', 'plt', 

In [9]:
print(os.getenv("TRANSFORMERS_CACHE"))  # Powinno zwrócić: D:\huggingface_cache

I:\HuggingFaceCache


# Model

In [6]:
from transformers import LongformerTokenizer, LongformerModel
import torch

In [8]:
start_time = time.time()

#tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-large-4096')
#model = LongformerModel.from_pretrained('allenai/longformer-large-4096')

#model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('intfloat/multilingual-e5-small')
#model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#model = SentenceTransformer('BAAI/bge-multilingual-gemma2') 18gb
#model = SentenceTransformer('Alibaba-NLP/gte-multilingual-base', trust_remote_code=True)
#model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

end_time = time.time()
print(f"Czas ładowania modelu: {end_time - start_time:.2f} sekundy")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

I:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sikma\.cache\huggingface\hub\models--intfloat--multilingual-e5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

Czas ładowania modelu: 124.97 sekundy


In [5]:
# Funkcja do konwersji tekstu na wektor (embedding)
def get_embedding(text):
    return model.encode(text)

def get_similarity(user_input, df_dataset):
    # Wektor wypowiedzi użytkownika
    user_embedding = get_embedding(user_input)
    
    # Oblicz podobieństwo dla każdej wypowiedzi w DataFrame
    df_dataset["similarity"] = df_dataset["embedding"].apply(lambda emb: cosine_similarity([user_embedding], [emb])[0][0])
    # Posortuj według podobieństwa malejąco
    df_sorted = df_dataset.sort_values(by="similarity", ascending=False)

    top_n = 5  # Weź 5 najbliższych wypowiedzi
    top_similar = df_sorted.head(top_n)

    if top_similar.iloc[0]["similarity"] > 0.9:
        avg_score = top_similar.iloc[0]["score"]
    else:
        avg_score = top_similar["score"].mean()
    return avg_score, top_similar[["similarity", "score", "statement"]].values.tolist()

In [15]:
def get_similarity_upgraded(user_input, df_dataset):
    user_embedding = get_embedding(user_input)
    
    df_dataset["similarity"] = df_dataset["embedding"].apply(lambda emb: cosine_similarity([user_embedding], [emb])[0][0])
    df_sorted = df_dataset.sort_values(by="similarity", ascending=False)

    top_n = 5
    top_similar = df_sorted.head(top_n)

    if top_similar.iloc[0]["similarity"] > 0.9:
        avg_score = top_similar.iloc[0]["score"]
    else:
        avg_score = top_similar["score"].mean()
        lower_bound = max(avg_score - 0.8, -1.0)
        upper_bound = min(avg_score + 0.8, 1.0)

        df_dataset_filtered = df_dataset[(df_dataset["score"] >= lower_bound) & (df_dataset["score"] <= upper_bound)]
        df_dataset_filtered = df_dataset_filtered.sort_values(by="similarity", ascending=False)
        top_similar = df_dataset_filtered.head(top_n)
        avg_score = top_similar["score"].mean()
        
    return avg_score, top_similar[["similarity", "score", "statement"]].values.tolist()


In [128]:
import math

def tanh_scaling(x, k=1.4):
    return math.tanh(k * x)

def power_scaling(x, b=0.5):
    return math.copysign(abs(x) ** b, x)

def log_scaling(x):
    return math.copysign(math.log(1 + abs(x)), x)

tanh_result = tanh_scaling(predicted_score)
power_result = power_scaling(predicted_score)
log_result = log_scaling(predicted_score)

print(f"Original value: {predicted_score}")
print(f"Tanh scaling: {tanh_result}")
print(f"Power scaling: {power_result}")
print(f"Log scaling: {log_result}")

NameError: name 'predicted_score' is not defined

# Funkcjonalność kompasu

In [8]:
pacyfizm_militaryzm = ['bron', 'obronnosc', 'sluzba_wojskowa']
nacjonalizm_kosmopolityzm = ['obronnosc', 'sluzba_wojskowa', 'armia_ue', 'euro', 'cpk', 'ue', 'imigranci']
ekologia_industrializm = ['samochody', 'cpk']
eurofederalizm_eurosceptyzm = ['euro', 'armia_ue', 'ue', 'samochody']
progresywizm_tradycjonalizm = ['aborcja', 'eutanazja', 'invitro', 'kara_smierci', 'bron']
socjalizm_liberalizm = ['osiemset', 'zus', 'dochodowy', 'katastralny', 'wdowia']
regulacjonizm_leseferyzm =  ['osiemset', 'zus', 'dochodowy', 'katastralny', 'wdowia']

In [9]:
class User:
    def __init__(self):
        self.pacyfizm_militaryzm_score = 0
        self.pacyfizm_militaryzm_answers = 0
        
        self.nacjonalizm_kosmopolityzm_score = 0
        self.nacjonalizm_kosmopolityzm_answers = 0

        self.ekologia_industrializm_score = 0
        self.ekologia_industrializm_answers = 0

        self.eurofederalizm_eurosceptyzm_score = 0        
        self.eurofederalizm_eurosceptyzm_answers = 0

        self.progresywizm_tradycjonalizm_score = 0        
        self.progresywizm_tradycjonalizm_answers = 0

        self.socjalizm_liberalizm_score = 0        
        self.socjalizm_liberalizm_answers = 0

        self.regulacjonizm_leseferyzm_score = 0        
        self.regulacjonizm_leseferyzm_answers = 0

        self.nolan_gospodarka_score = 0
        self.nolan_gospodarka_answers = 0

        self.nolan_obyczajowe_score = 0
        self.nolan_obyczajowe_answers = 0

    def add_score_to_compass(self, user_statement, chosen_topic, chosen_dataset):
        user_score, similiar_results = get_similarity_upgraded(user_statement, chosen_dataset)

        if chosen_topic in socjalizm_liberalizm or chosen_topic in regulacjonizm_leseferyzm:
            if chosen_topic in socjalizm_liberalizm:
                self.socjalizm_liberalizm_score += user_score
                self.socjalizm_liberalizm_answers += 1   
                
            if chosen_topic in regulacjonizm_leseferyzm:
                self.regulacjonizm_leseferyzm_score += user_score
                self.regulacjonizm_leseferyzm_answers += 1   
                
            self.nolan_gospodarka_score += user_score
            self.nolan_gospodarka_answers += 1
            
        else:  
            if chosen_topic in pacyfizm_militaryzm:       
                self.pacyfizm_militaryzm_score += user_score
                self.pacyfizm_militaryzm_answers += 1   
    
            if chosen_topic in nacjonalizm_kosmopolityzm:        
                self.nacjonalizm_kosmopolityzm_score += user_score
                self.nacjonalizm_kosmopolityzm_answers += 1   
                
            if chosen_topic in ekologia_industrializm:        
                self.ekologia_industrializm_score += user_score
                self.ekologia_industrializm_answers += 1   
                
            if chosen_topic in eurofederalizm_eurosceptyzm:       
                self.eurofederalizm_eurosceptyzm_score += user_score
                self.eurofederalizm_eurosceptyzm_answers += 1   
                
            if chosen_topic in progresywizm_tradycjonalizm:
                self.progresywizm_tradycjonalizm_score += user_score
                self.progresywizm_tradycjonalizm_answers += 1
                
            self.nolan_obyczajowe_score += user_score
            self.nolan_obyczajowe_answers += 1


    def display_scores(self):
        if self.pacyfizm_militaryzm_answers != 0:
            print(f'Wartość dla pacyfizm-militaryzm wynosi {self.pacyfizm_militaryzm_score/self.pacyfizm_militaryzm_answers}')
        if self.nacjonalizm_kosmopolityzm_answers != 0:
            print(f'Wartość dla nacjonalizm-kosmopolityzm wynosi {self.nacjonalizm_kosmopolityzm_score/self.nacjonalizm_kosmopolityzm_answers}')
        if self.ekologia_industrializm_answers != 0:
            print(f'Wartość dla ekologia-industrializm wynosi {self.ekologia_industrializm_score/self.ekologia_industrializm_answers}')
        if self.eurofederalizm_eurosceptyzm_answers != 0:
            print(f'Wartość dla eurofederalizm-eurosceptyzm wynosi {self.eurofederalizm_eurosceptyzm_score/self.eurofederalizm_eurosceptyzm_answers}')
        if self.progresywizm_tradycjonalizm_answers != 0:
            print(f'Wartość dla progresywizm-tradycjonalizm wynosi {self.progresywizm_tradycjonalizm_score/self.progresywizm_tradycjonalizm_answers}')
        if self.socjalizm_liberalizm_answers != 0:
            print(f'Wartość dla socjalizm-liberalizm wynosi {self.socjalizm_liberalizm_score/self.socjalizm_liberalizm_answers}')
        if self.regulacjonizm_leseferyzm_answers != 0:
            print(f'Wartość dla regulacjonizm-leseferyzm wynosi {self.regulacjonizm_leseferyzm_score/self.regulacjonizm_leseferyzm_answers}')
        
        if self.nolan_obyczajowe_answers != 0:
            print(f'Wartość dla diagramu Nolana konserwatyzm-liberalizm wynosi {self.nolan_obyczajowe_score/self.nolan_obyczajowe_answers}')
        if self.nolan_gospodarka_answers != 0:
            print(f'Wartość dla diagramu Nolana socjalizm-wolny rynek wynosi {self.nolan_gospodarka_score/self.nolan_gospodarka_answers}')

In [18]:
user = User()

In [19]:
for filename in os.listdir("datasets_embeddings"):
    filepath = os.path.join("datasets_embeddings", filename)

    topic = pd.read_csv(filepath, encoding="utf-8")
    topic["embedding"] = topic["embedding"].apply(lambda x: np.array(list(map(float, x.split(",")))))

    topic_name = f'{filename.replace("_embeddings.csv", "").replace("dataset_", "")}'
    print(topic_name)

aborcja
armia_ue
bron
cpk
dochodowy
euro
eutanazja
imigranci
invitro
kara_smierci
katastralny
osiemset
samochody
sluzba_wojskowa
ue
wdowia
zus


In [21]:
chosen_topic = input()
chosen_dataset_name = f'df_dataset_{chosen_topic}'
if chosen_dataset_name in globals():
    chosen_dataset = globals()[chosen_dataset_name]
    print(f"Dataset o nazwie {chosen_dataset_name} ZNALEZIONY.")
else:
    print(f"Dataset o nazwie {chosen_dataset_name} NIE ISTNIEJE.")

 bron


Dataset o nazwie df_dataset_bron ZNALEZIONY.


In [24]:
user_statement = input()

 Ogólnie jestem za łatwiejszym dostępem do posiadania broni palnej, bo to oznacza większe bezpieczeństwo jednostki, ale nie chciałbym mieć u nas drugiego USA. Restrykcje i testy powinny dalej istnieć.


In [114]:
user.add_score_to_compass(user_statement, chosen_topic, chosen_dataset)

In [115]:
user.display_scores()

Wartość dla pacyfizm-militaryzm wynosi 0.24000000000000002
Wartość dla nacjonalizm-kosmopolityzm wynosi 0.32
Wartość dla ekologia-industrializm wynosi 0.73
Wartość dla eurofederalizm-eurosceptyzm wynosi 0.34
Wartość dla progresywizm-tradycjonalizm wynosi 0.328
Wartość dla socjalizm-liberalizm wynosi 0.26
Wartość dla regulacjonizm-leseferyzm wynosi 0.26
Wartość dla diagramu Nolana konserwatyzm-liberalizm wynosi 0.35999999999999993
Wartość dla diagramu Nolana socjalizm-wolny rynek wynosi 0.26


In [25]:
avg_score, results = get_similarity(user_statement, chosen_dataset)

print("Średni score:", round(avg_score, 3))
print("Najbardziej podobne wypowiedzi:")
for sim, score, text in results:
    print(f"Similarity: {sim:.4f}, Score: {score}, Text: {text}")

Średni score: 0.34
Najbardziej podobne wypowiedzi:
Similarity: 0.7392, Score: 0.7, Text: Jestem za posiadaniem broni, ale pod warunkiem, że osoby ją kupujące przechodzą dokładne testy psychologiczne i odpowiednie szkolenia. Nie można dopuścić do sytuacji, w której ktoś niestabilny dostaje broń.
Similarity: 0.6888, Score: 0.1, Text: Nie uważam, żeby całkowity zakaz był konieczny, ale dostęp do broni powinien być dużo trudniejszy.
Similarity: 0.6840, Score: 0.1, Text: Obrona własna to jedno, ale broń nie powinna być zbyt łatwo dostępna.
Similarity: 0.6821, Score: 0.3, Text: Nie jestem wielkim fanem broni, ale rozumiem ludzi, którzy chcą jej mieć dla bezpieczeństwa.
Similarity: 0.6644, Score: 0.5, Text: Posiadanie broni powinno być przywilejem, a nie powszechnym prawem.


In [26]:
avg_score, results = get_similarity_upgraded(user_statement, chosen_dataset)

print(user_statement)
print('\n')
print("Średni score:", round(avg_score, 3))
print("Najbardziej podobne wypowiedzi:\n")
for sim, score, text in results:
    print(f"Similarity: {sim:.4f}, Score: {score}, Text: {text}\n")

Ogólnie jestem za łatwiejszym dostępem do posiadania broni palnej, bo to oznacza większe bezpieczeństwo jednostki, ale nie chciałbym mieć u nas drugiego USA. Restrykcje i testy powinny dalej istnieć.


Średni score: 0.34
Najbardziej podobne wypowiedzi:

Similarity: 0.7392, Score: 0.7, Text: Jestem za posiadaniem broni, ale pod warunkiem, że osoby ją kupujące przechodzą dokładne testy psychologiczne i odpowiednie szkolenia. Nie można dopuścić do sytuacji, w której ktoś niestabilny dostaje broń.

Similarity: 0.6888, Score: 0.1, Text: Nie uważam, żeby całkowity zakaz był konieczny, ale dostęp do broni powinien być dużo trudniejszy.

Similarity: 0.6840, Score: 0.1, Text: Obrona własna to jedno, ale broń nie powinna być zbyt łatwo dostępna.

Similarity: 0.6821, Score: 0.3, Text: Nie jestem wielkim fanem broni, ale rozumiem ludzi, którzy chcą jej mieć dla bezpieczeństwa.

Similarity: 0.6644, Score: 0.5, Text: Posiadanie broni powinno być przywilejem, a nie powszechnym prawem.



In [33]:
import math
avg_score= 0.1

def tanh_scaling(x, k=1.4):
    return math.tanh(k * x)

def power_scaling(x, b=0.5):
    return math.copysign(abs(x) ** b, x)

def log_scaling(x):
    return math.copysign(math.log(1 + abs(x)), x)
    
def amplify(x, scale=1):
    return math.tanh(x * scale) / math.tanh(scale)


tanh_result = tanh_scaling(avg_score)
power_result = power_scaling(avg_score)
log_result = log_scaling(avg_score)
amplify_result = amplify(avg_score)


print(f"Original value: {avg_score}")
print(f"Tanh scaling: {tanh_result}")
print(f"Power scaling: {power_result}")
print(f"Log scaling: {log_result}")
print(f"Aplify scaling: {amplify_result}")

Original value: 0.1
Tanh scaling: 0.139092447878458
Power scaling: 0.31622776601683794
Log scaling: 0.09531017980432493
Aplify scaling: 0.1308675937775247


In [58]:
import numpy as np

for i in np.arange(0, 1, 0.05):
    print(f'Dla {i} wzmocnienie wynosi {amplify(i):.2f}, wzrost o {(amplify(i)-i):.2f}')

Dla 0.0 wzmocnienie wynosi 0.00, wzrost o 0.00
Dla 0.05 wzmocnienie wynosi 0.07, wzrost o 0.02
Dla 0.1 wzmocnienie wynosi 0.13, wzrost o 0.03
Dla 0.15000000000000002 wzmocnienie wynosi 0.20, wzrost o 0.05
Dla 0.2 wzmocnienie wynosi 0.26, wzrost o 0.06
Dla 0.25 wzmocnienie wynosi 0.32, wzrost o 0.07
Dla 0.30000000000000004 wzmocnienie wynosi 0.38, wzrost o 0.08
Dla 0.35000000000000003 wzmocnienie wynosi 0.44, wzrost o 0.09
Dla 0.4 wzmocnienie wynosi 0.50, wzrost o 0.10
Dla 0.45 wzmocnienie wynosi 0.55, wzrost o 0.10
Dla 0.5 wzmocnienie wynosi 0.61, wzrost o 0.11
Dla 0.55 wzmocnienie wynosi 0.66, wzrost o 0.11
Dla 0.6000000000000001 wzmocnienie wynosi 0.71, wzrost o 0.11
Dla 0.65 wzmocnienie wynosi 0.75, wzrost o 0.10
Dla 0.7000000000000001 wzmocnienie wynosi 0.79, wzrost o 0.09
Dla 0.75 wzmocnienie wynosi 0.83, wzrost o 0.08
Dla 0.8 wzmocnienie wynosi 0.87, wzrost o 0.07
Dla 0.8500000000000001 wzmocnienie wynosi 0.91, wzrost o 0.06
Dla 0.9 wzmocnienie wynosi 0.94, wzrost o 0.04
Dla 0.95